### **RecursiveJsonSplitter란?**  

`RecursiveJsonSplitter`는 JSON 데이터를 **깊이 우선 탐색(DFS)**하여 더 작은 JSON 청크로 분할하는 도구임. 중첩된 JSON 구조를 최대한 유지하면서도, **청크 크기를 조정**하여 효율적인 처리가 가능하도록 함.  

---

### **🧐 왜 필요할까?**  

✔ **대형 JSON 처리 최적화** – JSON 데이터를 적절한 크기로 나누어 분석 및 저장이 용이함.  
✔ **중첩 구조 유지** – 가능한 한 원래 JSON 형태를 유지하여 데이터의 의미를 보존함.  
✔ **청크 크기 조절 가능** – `min_chunk_size`와 `max_chunk_size`를 설정하여 최적의 크기로 조정할 수 있음.  

---

### **🔍 어떻게 작동할까?**  

1️⃣ **JSON 데이터를 깊이 우선 탐색(DFS) 방식으로 순회**  
2️⃣ **설정된 청크 크기(`min_chunk_size`, `max_chunk_size`)를 고려하여 분할**  
3️⃣ **가능하면 원래 JSON 구조를 유지, 불가능할 경우 객체를 분리**  

예를 들어, 아래와 같은 JSON이 있다면:  

```json
{
  "name": "Example",
  "description": "이 JSON은 분할을 테스트하기 위한 예제입니다.",
  "data": {
    "items": [
      {"id": 1, "value": "데이터1"},
      {"id": 2, "value": "데이터2"}
    ]
  }
}
```

📌 **RecursiveJsonSplitter를 사용하면 다음과 같이 분할됨.**  

1️⃣ **첫 번째 청크** → `{ "name": "Example", "description": "이 JSON은..." }`  
2️⃣ **두 번째 청크** → `{ "data": { "items": [{"id": 1, "value": "데이터1"}...] } }`  

---

### **🛠 어디에 활용할 수 있을까?**  

✅ **API 데이터 처리** – 대형 JSON 데이터를 분할하여 효율적으로 저장 및 처리 가능.  
✅ **AI 및 NLP 분석** – AI 모델이 JSON 데이터를 분석하기 쉽게 분할하여 활용 가능.  
✅ **대용량 로그 데이터 처리** – 시스템 로그 JSON 파일을 분석하기 용이한 형태로 나눌 수 있음.  

---

### **🎯 한 줄 요약**  

**RecursiveJsonSplitter**는 JSON 데이터를 **깊이 우선 탐색 방식으로 적절한 크기의 청크로 나누는 도구**임. JSON의 중첩 구조를 유지하면서도 효율적인 분석·처리를 가능하게 함. 🚀


In [ ]:
import requests

# JSON 데이터를 로드함
json_data = requests.get("https://api.smith.langchain.com/openapi.json").json()

In [ ]:
json_data

`RecursiveJsonSplitter`를 사용하여 JSON 데이터를 분할


In [ ]:
from langchain_text_splitters import RecursiveJsonSplitter

# JSON 데이터를 최대 300 크기의 청크로 분할하는 RecursiveJsonSplitter 객체를 생성함
splitter = RecursiveJsonSplitter(max_chunk_size=300)

`splitter.split_json()` 함수를 사용하여 JSON 데이터를 재귀적으로 분할함


In [ ]:
# JSON 데이터를 재귀적으로 분할함 작은 JSON 조각에 접근하거나 조작해야 하는 경우에 사용함
json_chunks = splitter.split_json(json_data=json_data)

- `splitter.create_documents()` 메서드를 사용하여 JSON 데이터를 문서 형식으로 변환함
- `splitter.split_text()` 메서드를 사용하여 JSON 데이터를 문자열 리스트로 분할함


In [ ]:
# JSON 데이터를 기반으로 문서를 생성함
docs = splitter.create_documents(texts=[json_data])

# JSON 데이터를 기반으로 문자열 청크를 생성함
texts = splitter.split_text(json_data=json_data)

# 첫 번째 문자열을 출력함
print(docs[0].page_content)

print("===" * 20)
# 분할된 문자열 청크를 출력함
print(texts[0])

`texts[2]`을 출력하여 큰 청크 중 하나를 검토한 결과, 해당 청크에 리스트 객체가 포함되어 있음을 확인할 수 있음

- 2번째 청크의 크기가 제한(300) 을 초과하는 데에는 리스트 객체인 이유가 있음
- 이는 `RecursiveJsonSplitter` 가 **리스트 객체는 분할하지 않기 때문** 


In [ ]:
# 청크의 크기를 확인해 봅시다.
print([len(text) for text in texts][:10])

# 더 큰 청크 중 하나를 검토해 보면 리스트 객체가 있는 것을 볼 수 있음
print(texts[1])

2번 index 청크를 다음과 같이 `json` 모듈을 사용하여 파싱할 수 있음


In [ ]:
import json

json_data = json.loads(texts[2])
json_data["paths"]

`convert_lists` 매개변수를 `True`로 설정하여 JSON 내의 리스트를 `index:item` 형태의 `key:value` 쌍으로 변환할 수 있음


In [ ]:
# 다음은 JSON을 전처리하고 리스트를 인덱스:항목을 키:값 쌍으로 하는 딕셔너리로 변환함
texts = splitter.split_text(json_data=json_data, convert_lists=True)

In [ ]:
# 리스트가 딕셔너리로 변환되었고, 그 결과를 확인함
print(texts[2])

`docs` 리스트의 특정 인덱스에 해당하는 문서를 확인할 수 있음


In [ ]:
# 2번 문서를 확인함
docs[2]